In [ ]:
import numpy as np
import os
import json
from shapely.geometry import Polygon
import pandas as pd
import itertools
import tifffile as tiff
from skimage import measure
from tqdm import tqdm
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

In [ ]:
import numpy as np
import os
import json
from shapely.geometry import Polygon
from skimage import measure
from tqdm import tqdm
import itertools
import tifffile as tiff
import concurrent.futures  # Ensure this import is present

In [ ]:
import multiprocessing as mp

mp.set_start_method('spawn', force=True)


In [ ]:
import torch
from segment_anything import sam_model_registry

# Define model type and device
model_type = "vit_b"
device = "cuda" if torch.cuda.is_available() else "cpu"



sam = sam_model_registry[model_type](checkpoint="/kaggle/input/segment-anything/pytorch/vit-b/1/model.pth")
sam.to(device=device)

print(f"Model loaded on {device}")


In [ ]:
# Function to extract polygons from segmentation masks
def extract_polygons(segmentation, num_points_threshold, min_area):
    contours = measure.find_contours(segmentation, 0.5)
    contours = [contour for contour in contours if len(contour) >= num_points_threshold]
    contours = [contour for contour in contours if Polygon(contour).area >= min_area]
    polygons = []
    for contour in contours:
        polygon = []
        for point in contour:
            polygon.append(float(point[1]))  # X coordinate
            polygon.append(float(point[0]))  # Y coordinate
        polygons.append(polygon)
    return polygons

In [ ]:
# Function to save results to a JSON file
def save_results_to_json(results, output_file):
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=4)

In [ ]:

def process_image(image_path, combo, num_points_threshold, min_area):
    try:
        image = tiff.imread(image_path)
        band_red = image[:, :, combo[0]]
        band_green = image[:, :, combo[1]]
        band_blue = image[:, :, combo[2]]

        rgb_image = np.dstack((band_red, band_green, band_blue))
        image = (rgb_image - np.min(rgb_image)) / (np.max(rgb_image) - np.min(rgb_image))

        image = (image * 255).astype(np.uint8)

        segmentations = mask_generator.generate(np.array(image))

        annotations = []
        for segmentation in segmentations:
            polygons = extract_polygons(segmentation['segmentation'], num_points_threshold, min_area)
            for polygon in polygons:
                annotations.append({
                    "class": "field",
                    "segmentation": polygon
                })
        return {
            "file_name": os.path.basename(image_path),
            "annotations": annotations
        }
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return {
            "file_name": os.path.basename(image_path),
            "annotations": []
        }


In [ ]:
import os
import itertools
from tqdm import tqdm

# Define the top 5 channel combinations
top_combinations = [
    (1, 3, 6),




]

def process_images(image_dir, output_dir, num_points_threshold, min_area):
    for combo in top_combinations:
        results = {"images": []}

        image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if filename.endswith('.tif')]

        for image_path in tqdm(image_paths, desc=f"Processing combo {combo}"):
            result = process_image(image_path, combo, num_points_threshold, min_area)
            results["images"].append(result)

        output_file = os.path.join(output_dir, f'segmentation_results_combo_{combo[0]}_{combo[1]}_{combo[2]}.json')
        save_results_to_json(results, output_file)
        print(f"Results saved to {output_file}")



In [ ]:
# Mask generator settings
mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=65,
    pred_iou_thresh=0.91,
    stability_score_thresh=0.97,
    crop_n_layers=2,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=120,  # Minimum area to consider a valid segmentation
    

)


In [ ]:
# Set parameters
image_dir = '/kaggle/input/field-area-segmentation-3/Field_area_segmentation/train_images/images'
output_dir = '/kaggle/working/'
num_points_threshold = 50  # Minimum number of points in a contour to be considered
min_area = 100  # Minimum area of a polygon to be considered

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process images and save results
process_images(image_dir, output_dir, num_points_threshold, min_area)

In [ ]:
def getIOU(polygon1: Polygon, polygon2: Polygon):
    intersection = polygon1.intersection(polygon2).area
    union = polygon1.union(polygon2).area
    if union == 0:
        return 0
    return intersection / union


def compute_pq(gt_polygons: list, pred_polygons: list, iou_threshold=0.5):
    matched_instances = {}
    gt_matched = np.zeros(len(gt_polygons))
    pred_matched = np.zeros(len(pred_polygons))

    gt_matched = np.zeros(len(gt_polygons))
    pred_matched = np.zeros(len(pred_polygons))
    for gt_idx, gt_polygon in enumerate(gt_polygons):
        best_iou = iou_threshold
        best_pred_idx = None
        for pred_idx, pred_polygon in enumerate(pred_polygons):
            # if gt_matched[gt_idx] == 1 or pred_matched[pred_idx] == 1:
            #     continue
            try:
                iou = getIOU(gt_polygon, pred_polygon)
            except:
                iou = 0
                print('Error Polygon -> iou is 0')

            if iou == 0:
                continue

            if iou > best_iou:
                best_iou = iou
                best_pred_idx = pred_idx
        if best_pred_idx is not None:
            matched_instances[(gt_idx, best_pred_idx)] = best_iou
            gt_matched[gt_idx] = 1
            pred_matched[best_pred_idx] = 1


    sq_sum = sum(matched_instances.values())
    num_matches = len(matched_instances)
    sq = sq_sum / num_matches if num_matches else 0
    rq = num_matches / ((len(gt_polygons) + len(pred_polygons))/2.0) if (gt_polygons or pred_polygons) else 0
    pq = sq * rq

    return pq, sq, rq

In [ ]:
import os
import json
import pandas as pd
from shapely.geometry import Polygon
from tqdm import tqdm

# Load ground truth annotations
with open('/kaggle/input/annotaion-json/train_annotation.json') as f:
    gts = json.load(f)

scores, files = [], []

# Iterate through files in the directory
for k in os.listdir('/kaggle/working/'):
    file_path = os.path.join('/kaggle/working/', k)

    # Check if the item is a file
    if os.path.isfile(file_path) and file_path.endswith('.json'):
        with open(file_path) as f:
            submits_json = json.load(f)

        for i, _image_pred in tqdm(enumerate(submits_json['images'])):
            fname = _image_pred['file_name']
            annos_pred = _image_pred['annotations']
            print(fname)

            # Find matching ground truth
            fname_gt = None
            _image_gt = next((img for img in gts['images'] if img['file_name'] == fname), None)
            if _image_gt is None:
                print(f'No ground truth found for file: {fname}')
                continue

            fname_gt = _image_gt['file_name']
            annos_gt = _image_gt['annotations']

            print(f'File:{fname} - {fname_gt} Num GT: {len(annos_gt)}, Num Pred: {len(annos_pred)}')

            polygons_gt, polygons_pred = [], []
            for anno in annos_gt:
                _polys = [(x, y) for x, y in zip(anno['segmentation'][::2], anno['segmentation'][1::2])]
                polygons_gt.append(Polygon(_polys))

            for anno in annos_pred:
                _polys = [(x, y) for x, y in zip(anno['segmentation'][::2], anno['segmentation'][1::2])]
                polygons_pred.append(Polygon(_polys))

            pq, sq, rq = compute_pq(polygons_gt, polygons_pred)
            print(f'File:{fname} PQ: {pq:.4f}, SQ: {sq:.4f}, RQ: {rq:.4f} Num: {len(polygons_gt)}')

            scores.append([pq, sq, rq])
            files.append(fname)

        df = pd.DataFrame(scores, columns=['PQ', 'SQ', 'RQ'], index=files)
        df['file'] = files

        mean_pq = df['PQ'].mean()
        print(f'Mean PQ: {mean_pq:.4f}')
        df.to_csv(f'exp_{k}.csv')


In [ ]:
# Folder containing the CSV files
folder_path = '/kaggle/working/'

# Initialize lists to store results
combos = []
mean_pqs = []
mean_sqs = []
mean_rqs = []

# Process each file in the folder
for file_name in os.listdir(folder_path)[3:]:
    if file_name.endswith('.csv'):
        # Extract combo from file name
        combo = '(' + file_name.split('_')[-3] + '_' + file_name.split('_')[-2] + '_' + file_name.split('_')[-1].replace('.json.csv', '')+ ')'
        combo = combo.replace("_", " ")

        # Read the CSV file
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)

        # Calculate means
        mean_pq = df['PQ'].mean()
        mean_sq = df['SQ'].mean()
        mean_rq = df['RQ'].mean()

        # Append results to lists
        combos.append(combo)
        mean_pqs.append(mean_pq)
        mean_sqs.append(mean_sq)
        mean_rqs.append(mean_rq)

# Create a new DataFrame with the results
result_df = pd.DataFrame({
    'Channels': combos,
    'mean_pq': mean_pqs,
    'mean_sq': mean_sqs,
    'mean_rq': mean_rqs
})

df_sorted = result_df.sort_values(by='mean_pq', ascending=False)

# Select the top 20 values
df_sorted.head()

In [ ]:
def extract_polygons(segmentation, num_points_threshold, min_area):
    contours = measure.find_contours(segmentation, 0.5)
    contours = [contour for contour in contours if len(contour) >= num_points_threshold]
    contours = [contour for contour in contours if Polygon(contour).area >= min_area]
    polygons = []
    for contour in contours:
        polygon = []
        for point in contour:
            polygon.append(float(point[1]))  # X coordinate
            polygon.append(float(point[0]))  # Y coordinate
        polygons.append(polygon)
    return polygons

In [ ]:
import os
import numpy as np
from shapely.geometry import Polygon
from tqdm import tqdm
from PIL import Image

def process_images(image_dir, num_points_threshold, min_area):
    images = []
    for filename in tqdm(os.listdir(image_dir)):
        if filename.endswith('.tif'):
            image_path = os.path.join(image_dir, filename)
            image = tiff.imread(image_path)
            band_red = image[:, :, 1]
            band_green = image[:, :, 3]
            band_blue = image[:, :, 6]

            rgb_image = np.dstack((band_red, band_green, band_blue))

            # Normalize the image to 0-255 and convert to uint8
            image = (rgb_image - np.min(rgb_image)) / (np.max(rgb_image) - np.min(rgb_image))
            image = (image * 255).astype(np.uint8)

            # Ensure the image is in HWC format
            if image.shape[2] == 1:  # If single channel, convert to RGB
                image = np.stack([image] * 3, axis=-1)

            segmentations = mask_generator.generate(image)  # Replace with actual function call to your segmentation model

            annotations = []
            for segmentation in segmentations:
                polygons = extract_polygons(segmentation['segmentation'], num_points_threshold, min_area)
                for polygon in polygons:
                    annotations.append({
                        "class": "field",
                        "segmentation": polygon
                    })
            images.append({
                "file_name": filename,
                "annotations": annotations
            })
    return {"images": images}


In [ ]:
image_dir ='/kaggle/input/field-area-segmentation-3/Field_area_segmentation/test_images/images'

output_file = 'results.json'
Number_of_point = 3
size = 10

# Process images and save results
results = process_images(image_dir, Number_of_point, size)
save_results_to_json(results, output_file)
print(f"Results saved to {output_file}")

In [ ]:
with open('/kaggle/input/annotaion-json/train_annotation.json') as f:
    gts = json.load(f)

with open('results.json') as f:
    val_json = json.load(f)

scores, files = [], []

for i ,(_image_pred) in enumerate(val_json['images']):

    fname = _image_pred['file_name']
    annos_pred = _image_pred['annotations']
    print(fname)
    for j ,(_image_gt) in enumerate(gts['images']):
        if _image_gt['file_name'] == fname:
            fname_gt = _image_gt['file_name']
            break

    annos_gt = _image_gt['annotations']

    print(f'File:{fname} - {fname_gt} Num GT: {len(annos_gt)}, Num Pred: {len(annos_pred)}')

    polygons_gt, polygons_pred = [], []
    for anno in annos_gt:
        _polys = []
        for ii, (x, y) in enumerate(zip(anno['segmentation'][::2], anno['segmentation'][1::2])):
            _polys.append((x, y))
        polygons_gt.append(Polygon(_polys))

    for anno in annos_pred:
        _polys = []
        for ii, (x, y) in enumerate(zip(anno['segmentation'][::2], anno['segmentation'][1::2])):
            _polys.append((x, y))
        polygons_pred.append(Polygon(_polys))


    pq, sq, rq = compute_pq(polygons_gt, polygons_pred)
    print(f'File:{fname} PQ: {pq:.4f}, SQ: {sq:.4f}, RQ: {rq:.4f} Num: {len(polygons_gt)}')

    scores.append([pq, sq, rq])
    files.append(fname)

df = pd.DataFrame(scores, columns=['PQ', 'SQ', 'RQ'], index=files)
df['file'] = files

metrisc = df['PQ'].mean()
print(f'Mean PQ: {metrisc:.4f}')